In [1]:
import sqlite3
import pandas as pd
from flask import Flask
from flask import Markup
from flask import Flask, request, render_template,send_file, Response, redirect, url_for, jsonify, send_file, session
from werkzeug.wrappers import Request, Response
from werkzeug.serving import run_simple
import json
from dateutil.parser import parse
import os
import database_helper as db_helper
import csvScript
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user, fresh_login_required
from urllib.parse import urlparse, urljoin
from werkzeug.security import generate_password_hash, check_password_hash
from flask_mail import Mail, Message

In [9]:
csvScript.runScript()

student [caldwell,ivan alexander] is already in the student table
student [robenalt, evan] is already in the student table
student [chua,andreana] is already in the student table


Student [Caldwell,Ivan Alexander] already in Mentoring table. (student has already been assigned).


Student [Robenalt, Evan] already in Mentoring table. (student has already been assigned).
Student [Chua,Andreana] already in Mentoring table. (student has already been assigned).

In [2]:
app = Flask(__name__)

In [3]:
app.config['TEMPLATES_AUTO_RELOAD'] = True			# If template change, there is no need to reload the APP
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0			# No caching, only for debugging purposes
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = True

THIS_FOLDER = os.path.abspath('')
db_file = os.path.join(THIS_FOLDER, 'login.db')   
db_file = "sqlite:///" + db_file
fileStr = db_file

app.config['SQLALCHEMY_DATABASE_URI'] = fileStr
app.config['SECRET_KEY'] = 'da17fb86e4a1121551ac7062aa830e41'
#app.config['USE_SESSION_FOR_NEXT'] = True

db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
login_manager.login_message = "You need to login!"
login_manager.refresh_view ='login'
login_manager.need_refresh_message = 'You need to re-log in to refresh this page'

class User(UserMixin,db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(30),unique=True)
    password = db.Column(db.String(80))
    
def is_safe_url(target):
    ref_url = urlparse(request.host_url)
    test_url = urlparse(urljoin(request.host_url,target))
    return test_url.scheme in ('http','https') and \
        ref_url.netloc == test_url.netloc  


@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))    

In [4]:
app.config['MAIL_SERVER'] = "smtp.gmail.com"
app.config['MAIL_PORT'] =  587
app.config['MAIL_USERNAME'] = "cs.sys.testing@gmail.com"
app.config['MAIL_PASSWORD'] =  "a474561939"
app.config['MAIL_USE_TLS'] = True
mail = Mail(app)

In [5]:
@app.route('/')
def index():
    return redirect(url_for("login"))

@app.route('/login')
def login():
    if(current_user.is_authenticated == False):
        session['next'] = request.args.get('next', "/main_menu" )
        return render_template('login.html')        
    else:
        return redirect(url_for("mainMenu"))

@app.route('/login_request',methods=['POST','GET'])
def login_request():
    username = request.form['username']
    form_password = request.form['password']
    user = User.query.filter_by(username=username).first()
    if not user:
        return jsonify({'RESULT':'fail','MSG':'There is no such user.'})
    
    if check_password_hash(user.password, form_password):
        login_user(user,remember=True)
    
        if 'next' in session:
            next = session['next']
            
            if is_safe_url(next):
                return jsonify({'RESULT':'SUCCESS','MSG':next})
            else:
                return jsonify({'RESULT':'SUCCESS','MSG':'/main_menu'})
        else:
            return jsonify({'RESULT':'SUCCESS','MSG':'/main_menu'})
    else:
        return jsonify({'RESULT':'FAIL','MSG':'password incorrect.'})
        

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for("login"))

@app.route('/change_password')
@login_required
def change_password():
    return render_template("change_password.html")

@app.route('/change_password_request', methods=['POST','GET'])
def change_password_request():
    new_password = request.form['new_password'] 
    if(current_user.is_authenticated == True):      
        try:
            User.query.filter_by(username=current_user.username).update({"password":generate_password_hash(new_password)})
            db.session.commit()
            return jsonify({'RESULT':'SUCCESS','MSG':'/main_menu'})
        except Exception as e: 
            return jsonify({'RESULT':'FAIL','MSG':str(e)})
    else:
        return jsonify({'RESULT':'FAIL','MSG':'unable to locate user'})
        

# A route to show all numbers and the tables including the joined table
@app.route('/showall',methods = ['POST', 'GET'])
@login_required
def showall():
    return render_template("showall.html",
    mentoring_tables=[db_helper.readDatabaseIntoDataframe(connection).to_html(classes='data',index=False)], 
    student_table=[db_helper.readTableIntoDataFrame(connection,"student").to_html(classes='data',index=False)],
    professor_table=[db_helper.readTableIntoDataFrame(connection,"professor").to_html(classes='data',index=False)],
    mentor_table=[db_helper.readTableIntoDataFrame(connection,"mentoring").to_html(classes='data',index=False)],
    offline_data_table=[db_helper.readTableIntoDataFrame(connection,'offline_data').to_html(classes='data',index=False)], 
    most_recent_mentoring_updates_table=[db_helper.readTableIntoDataFrame(connection,'most_recent_mentoring_updates').to_html(classes='data',index=False)],                     
    numberForEachTable=db_helper.getAllNumbers(cursor),
    professorDictionary=db_helper.getProfessorAndStudentNumberInDicionary(cursor))

# The main route for this APP
@app.route('/main_menu',methods = ['POST', 'GET'])
@login_required
def mainMenu():
    professorAndNumberDict = db_helper.getProfessorAndStudentNumberInDicionary(cursor)
    professorJsonString = ""
    for professorName,count in sorted(professorAndNumberDict.items()):
        status = ""
        if count < db_helper.getAllNumbers(cursor)[3]:
            status = "EMPTY"
        elif count >= db_helper.getAllNumbers(cursor)[3] and count < db_helper.getAllNumbers(cursor)[4]:
            status = "ALMOST FULL"
        else:
            status = "FULL"
        professorJsonString += (json.dumps({
            "professorName":professorName.strip().title(),
            "count":count,
            'status': status
        })) + ","
    professorJsonString = '[' + professorJsonString[:-1] + ']'

    student_table = db_helper.readTableIntoDataFrame(connection,"student").sort_values(by=['lastname'])
    student_table['duplicated_name'] = student_table.duplicated(subset=['lastname','firstname'], keep=False)
    studentString = ""
    for i in range(student_table.shape[0]):
        duplicated = 0
        if student_table.iloc[i].duplicated_name == True:
            duplicated = 1
        studentString += (json.dumps({
            "studentName":student_table.iloc[i].lastname.title() + ", " + student_table.iloc[i].firstname.title(),
            "relationship_id":db_helper.findStudentIdByRelationshipId(cursor,student_table.iloc[i].ID),
            'email': student_table.iloc[i].email,
            'duplicated': str(duplicated)
        })) + ","
    studentString = '[' + studentString[:-1] + ']'    
    
    current_user_name  = ""
    if(current_user.is_authenticated == True):
        current_user_name = current_user.username

    return render_template("main_menu.html",
                           studentJson=studentString, 
                           professorJson=professorJsonString, 
                           numberForEachTable=db_helper.getAllNumbers(cursor), current_user_name = current_user_name)

# Update the main_menu page by selecting the element "student_name_input"
@app.route('/update',methods=['POST'])
def update():
    relationship_id = request.form['relationship_id'] 
    professorid = db_helper.findProfessorIdByMentoringId(cursor,relationship_id)
    if professorid is not 'null':
        professorname =  db_helper.getProfessorObjectById(cursor,professorid).lastname
    else:
        professorname =  "None"
    
    studentCountForThisProfessor = db_helper.getProfessorAndStudentNumberInDicionary(cursor)[professorname]
    if studentCountForThisProfessor < db_helper.getAllNumbers(cursor)[3]:
        color = "green"
    elif studentCountForThisProfessor >= db_helper.getAllNumbers(cursor)[3] and studentCountForThisProfessor < db_helper.getAllNumbers(cursor)[4]:
        color = "yellow"
    else:
        color = "red"
           
    return jsonify({'RESULT':'SUCCESS','professor_name':professorname.capitalize(),'student_count':db_helper.getProfessorAndStudentNumberInDicionary(cursor)[professorname],'number_color':color})

# Update the main_menu page by selecting the element "mentor_name_input"
@app.route('/update_from_professor_datalist',methods=['POST'])
def update_from_professor_datalist():
    selected = request.form['name']    
    professorname = selected.strip().lower()
    studentCountForThisProfessor = db_helper.getProfessorAndStudentNumberInDicionary(cursor)[professorname]
    if studentCountForThisProfessor < db_helper.getAllNumbers(cursor)[3]:
        color = "green"
    elif studentCountForThisProfessor >= db_helper.getAllNumbers(cursor)[3] and studentCountForThisProfessor < db_helper.getAllNumbers(cursor)[4]:
        color = "yellow"
    else:
        color = "red"
           
    return jsonify({'RESULT':'SUCCESS','student_count':db_helper.getProfessorAndStudentNumberInDicionary(cursor)[professorname],'number_color':color})


# UPDATE the mentoring table by the assign button
@app.route('/assign_mentor_existed_student',methods=['POST','GET'])
def assign_mentor_existed_student():
    studentId = globalMentoring.student_id
    professorId = globalMentoring.professor_id
    studentName = globalMentoring.studentName
    mentorName = globalMentoring.professorName
    assignDate = globalMentoring.assignedDate
    endDate = globalMentoring.endDate
    relationship_id = globalMentoring.relationship_id
    email_option_checked = request.form['email_option_checked']
    mail_sent = "true"    
    
    mentorEmail = db_helper.getProfessorObjectById(cursor,professorId).email
    studentRebelMail = db_helper.getStudentObjectById(cursor,studentId).email
    
    print("student " + studentName + " is assinged to mentor " + mentorName + " assigned date: " + assignDate + " end date " + endDate + ".")
    db_helper.assignExistedStudentToProfessorById(connection,studentId,professorId,assignDate,endDate)
    db_helper.insertDataToMost_recent_mentoring_updatesTable(connection,relationship_id) 
#     if email_option_checked == "true":
#         try:
#             msg = Message(subject="Mentor request",
#                           sender=app.config['MAIL_USERNAME'],
#                           recipients=[studentRebelMail],
#                           cc = [],
#                           body="Hi,\nYour faculty mentor is professor " + mentorName.title() + ".  Please email him at"  + mentorEmail + " to schedule an appointment.  Please note, before we can sign the advanced standing form, you will need to have completed your grad plan.  Thanks.\n\n-" + current_user.username)
#             mail.send(msg) 
#             mail_sent = "true"            
#         except Exception as e:
#             mail_sent = "false" 
#             print(str(e))
            
    return jsonify({'RESULT':'ASSIGNED','EMAIL_SENT':mail_sent})
            

@app.route('/assign_mentor_existed_student_check',methods=['POST','GET'])
def assign_mentor_existed_student_check():
    studentName = request.form['studentName'].lower()
    mentorName = request.form['mentorName'].lstrip().rstrip().lower()
    assignDate = request.form['assignDate']
    endDate = request.form['endDate']
    relationship_id = request.form['relationship_id']
    print("checking condition for: student " + studentName + " is assinging to mentor " + mentorName + " assigned date: " + assignDate + " end date " + endDate + ".")
    
    if len(studentName.split(',')) < 2:
        return jsonify({'RESULT':'Student name is not in correct format. For example: "Lastname, Firstname" '})
    
    studentId = db_helper.findStudentIdByRelationshipId(cursor,relationship_id)
    professorId = db_helper.findProfessorIDByName(cursor,mentorName,"")
    
    if studentId == "null":
        return jsonify({'RESULT':'Unable to find this student. Try create new student.'})
    
    if professorId == "null":
        return jsonify({'RESULT':"Unable to find this mentor. Please double check this mentor's name."})
    
    if professorId == db_helper.findProfessorByStudentId(cursor,studentId):
        return jsonify({'RESULT':'This student has already been assigned to this mentor.'})  
    
    globalMentoring.professor_id = professorId
    globalMentoring.student_id = studentId
    globalMentoring.assignedDate = assignDate
    globalMentoring.endDate = endDate
    globalMentoring.studentName = studentName
    globalMentoring.professorName = mentorName
    globalMentoring.relationship_id = relationship_id
    
    return jsonify({'RESULT':'SUCCESS'})
    
    
@app.route('/assign_mentor_new_stduent',methods=['POST','GET'])
def assign_mentor_new_stduent():
    professorId = globalMentoring.professor_id
    studentName = globalMentoring.studentName
    mentorName = globalMentoring.professorName
    assignDate = globalMentoring.assignedDate
    endDate = globalMentoring.endDate
    studentRebelMail = globalMentoring.studentRebelMail
    email_option_checked = request.form['email_option_checked']
    mail_sent = "true"  
    
    studentLastName = studentName.split(',')[0].rstrip().lstrip()
    studentFirstName = studentName.split(',')[1].rstrip().lstrip()
    
    mentorEmail = db_helper.getProfessorObjectById(cursor,professorId).email
    
    print("New student " + studentName + " is assinged to mentor " + mentorName + " assigned date: " + assignDate + " end date " + endDate + ".")
    studentId = db_helper.insertNewStudent(cursor,studentLastName,studentFirstName,studentRebelMail)
    print("Student id from insertNewStudent: " + str(studentId))
    db_helper.assignNewStudentToProfessorById(connection,studentId,professorId,assignDate,endDate)
    if email_option_checked == "true":
        try:
            msg = Message(subject="Mentor request",
                          sender=app.config['MAIL_USERNAME'],
                          recipients=[studentRebelMail],
                          cc = [],
                          body="Hi,\nYour faculty mentor is professor " + mentorName.title() + ".  Please email him at"  + mentorEmail + " to schedule an appointment.  Please note, before we can sign the advanced standing form, you will need to have completed your grad plan.  Thanks.\n\n-" + current_user.username)
            mail.send(msg) 
            mail_sent = "true"            
        except Exception as e:
            mail_sent = "false" 
            print(str(e))
    return jsonify({'RESULT':'ASSIGNED','EMAIL_SENT':mail_sent})    

@app.route('/assign_mentor_new_stduent_check',methods=['POST','GET'])
def assign_mentor_new_stduent_check():
    studentLastName = request.form['studentLastName'].rstrip().lstrip().lower()
    studentFirstName = request.form['studentFirstName'].rstrip().lstrip().lower()
    mentorName = request.form['mentorName'].rstrip().lstrip().lower()
    assignDate = request.form['assignDate']
    endDate = request.form['endDate']    
    studentRebelMail = request.form['studentRebelMail'].rstrip().lstrip().lower()
    studentName = studentLastName + ", " + studentFirstName

    print("Checking conditions for student  " + studentName + " is assinging to mentor " + mentorName + " assigned date: " + assignDate + " end date " + endDate + ".")
    
    student_id = db_helper.findStudentIDByNameAndMail(cursor,studentLastName,studentFirstName,studentRebelMail)
    if student_id != "null":
        return jsonify({'RESULT':"Student already existed. Try turn off the New Student Switch."})

    professorId = db_helper.findProfessorIDByName(cursor,mentorName,"")       
    if professorId == "null":
        return jsonify({'RESULT':"Unable to find this mentor. Please double check this mentor's name."})    

    globalMentoring.professor_id = professorId
    globalMentoring.assignedDate = assignDate
    globalMentoring.endDate = endDate
    globalMentoring.studentName = studentName
    globalMentoring.professorName = mentorName 
    globalMentoring.studentRebelMail = studentRebelMail
    
    return jsonify({'RESULT':'SUCCESS'})

# No caching, debugging purposes
@app.after_request
def add_header(r):
    """
    Add headers to both force latest IE rendering engine or Chrome Frame,
    and also to cache the rendered page for 10 minutes.
    """
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r

@app.route('/mentoring_joined',methods=['POST','GET'])
@login_required
def mentoring_joined():
    most_recent_changes_json_string = json.dumps(db_helper.getMost_recent_mentoring_updatesTableAsDataframeForFlask(connection).sort_index(ascending=False).values.tolist())
    professorAndNumberDict = db_helper.getProfessorAndStudentNumberInDicionary(cursor)
    newString = ""
    for professorName,count in sorted(professorAndNumberDict.items()):
        newString += (json.dumps({
            "professorName":professorName.strip().title(),
            "count":count
        })) + ","
    newString = '[' + newString[:-1] + ']'
    
    try:
        studentName = request.args.get('username').title()
    except:
        studentName = ""
        
    current_user_name  = ""
    if(current_user.is_authenticated == True):
        current_user_name = current_user.username
        
    return render_template("mentoring_joined.html",
        mentoring_joined_tables=[db_helper.readDatabaseIntoDataframe(connection,True).to_html(classes='data',index=False)],
        studentName = studentName,
        professorJson=newString,
        most_recent_changes_json_string = most_recent_changes_json_string, current_user_name = current_user_name                   
      )
  
@app.route('/apply_settings',methods=['POST'])   
def apply_settings():
    margin = float(request.form['margin'])
    db_helper.updateDataFromOffline_dataTable(connection,'DOUBLE_margin_for_max_student_count',margin)
    return jsonify({'RESULT':'SUCCESS'}) 

@app.route('/backup_database',methods=['POST','GET'])
@login_required
def backup_database():
    current_user_name  = ""
    if(current_user.is_authenticated == True):
        current_user_name = current_user.username       
        
    return render_template("backup_database.html",current_user_name=current_user_name) 

@app.route('/request_for_backup_database')   
def request_for_backup_database():
    THIS_FOLDER = os.path.dirname(os.path.abspath(__file__))
    db_file = os.path.join(THIS_FOLDER, "falcuty_mentor.db")
    try:
        file = send_file(db_file, as_attachment=True,attachment_filename='UNLV_CS_mentor.db') 
    except Exception as e:
        file = str(e)
    
    return file


@app.route('/multiple_assign_page',methods=['POST','GET'])
@login_required
def multiple_assign_page():
    professorAndNumberDict = db_helper.getProfessorAndStudentNumberInDicionary(cursor)
    newString = ""
    for professorName,count in sorted(professorAndNumberDict.items()):
        newString += (json.dumps({
            "professorName":professorName.strip().title(),
            "count":count
        })) + ","
    newString = '[' + newString[:-1] + ']'
    studentForProfessorsJson = db_helper.getStudentsForProfessor(connection)
    
    current_user_name  = ""    
    if(current_user.is_authenticated == True):
        current_user_name = current_user.username    
    
    return render_template("multiple_assign_page.html",professorJson=newString,studentForProfessorsJson=studentForProfessorsJson,current_user_name=current_user_name)

@app.route('/multiple_assign_request',methods=['POST','GET'])
def multiple_assign_request():
    teacherName = request.form['teacherName']
    relationshipIdArray = request.form.getlist('relationshipIdArray[]')
    db_helper.massAssign(connection,relationshipIdArray,teacherName)
    mail_sent = "true"
    mentor_id = db_helper.findProfessorIDByName(cursor,teacherName.lower(),"")
    mentorEmail = db_helper.getProfessorObjectById(cursor,mentor_id)
    
#     with mail.connect() as conn:
#         for i in relationshipIdArray:
#             student_id = db_helper.findStudentIdByRelationshipId(i)
#             studentRebelMail = db_helper.getStudentObjectById(student_id).email
#             try:
#                 msg = Message(subject="Mentor request",
#                               sender=app.config['MAIL_USERNAME'],
#                               recipients=[studentRebelMail],
#                               cc = [],
#                               body="Hi,\nYour faculty mentor is professor " + teacherName.title() + ".  Please email him at"  + mentorEmail + " to schedule an appointment.  Please note, before we can sign the advanced standing form, you will need to have completed your grad plan.  Thanks.\n\n-" + current_user.username)
#                 mail.send(msg) 
#                 mail_sent = "true"            
#             except Exception as e:
#                 mail_sent = "false" 
#                 print(str(e))
        
    return jsonify({'RESULT':'SUCCESS','EMAIL_SENT':mail_sent})

@app.route('/manage_mentors',methods=['POST','GET'])
@login_required
def manage_mentors():
    current_user_name  = ""    
    if(current_user.is_authenticated == True):
        current_user_name = current_user.username      
        
    df = db_helper.readTableIntoDataFrame(connection,'professor')
    df["Professor"] = df["lastname"].str.title().str.lstrip().str.rstrip()
    del df["lastname"]
    del df["firstname"]
    df = df.rename(columns={"email":"Professor Email"})
    col = ["ID","Professor","Professor Email"]
    df=df[col]
    df["Student Numbers"] = list(db_helper.getProfessorAndStudentNumberInDicionary(cursor).values())
    df.sort_values(by=["Professor"],inplace=True)
    
    return render_template("manage_mentors.html",current_user_name=current_user_name,mentor_2d_array = df.values)

@app.route('/update_mentor',methods=['POST','GET'])
def update_mentor():
    mentor_id = request.form['mentor_id']
    new_mentor_name = request.form['new_mentor_name']
    new_mentor_email = request.form['new_mentor_email']
    new_mentor_name = new_mentor_name.lstrip().rstrip().lower()
    new_mentor_email = new_mentor_email.lstrip().rstrip().lower()
    
    result = db_helper.updateMentor(connection,mentor_id,new_mentor_name,new_mentor_email)
    return jsonify({'RESULT':result[0],'MSG':result[1]})
    
@app.route('/delete_mentor',methods=['POST','GET'])
def delete_mentor():
    mentor_id = request.form['mentor_id']
    student_count = db_helper.getNumberOfStudentForMentorID(cursor,mentor_id)
    if student_count is not "null":
        if student_count > 0:
            return jsonify({'RESULT':"FAIL",'MSG':'please make sure this mentor has no students before deleting this mentor.'})
        else:
            result = db_helper.deleteProfessorById(connection,mentor_id)
            return jsonify({'RESULT':result[0],'MSG':result[1]})
    else:
        return jsonify({'RESULT':"FAIL",'MSG':'Unable to find mentor.'})
                
    

In [ ]:
# Running the app here
if __name__ == '__main__':
    connection = sqlite3.connect("falcuty_mentor.db")
    connection.execute("PRAGMA foreign_keys = ON;")
    cursor = connection.cursor()
    globalMentoring = db_helper.Mentoring()
    run_simple('localhost', 5000, app,use_debugger=True, use_evalex=True)


 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Apr/2020 02:56:17] "GET /manage_mentors HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 02:56:17] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:56:17] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:56:17] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:56:17] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:56:50] "POST /delete_mentor HTTP/1.1" 200 -


0


127.0.0.1 - - [04/Apr/2020 02:57:02] "GET /manage_mentors HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 02:57:02] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:02] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:02] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:02] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:10] "GET /multiple_assign_page HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 02:57:10] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:10] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:11] "GET /static/images/icons/success_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:11] "GET /static/table2CSV.js HTTP/1.1" 304 -


assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "1";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "22";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "2";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "3";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "23";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "4";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "5";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "6";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/202

assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "53";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "90";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "8";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "54";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "55";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "1087";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "1086";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "04/04/2020" WHERE ID = "56";
assignFromRelationshipId(): UPDATE mentoring SET professor_id = "3", dateAssigned = "

...

127.0.0.1 - - [04/Apr/2020 02:57:29] "GET /mentoring_joined HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 02:57:29] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:29] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:29] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:29] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:29] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:38] "GET /manage_mentors HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 02:57:39] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:39] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:39] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:39] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:46] "POST /delete_mentor HTTP/1.1" 200 -


0


127.0.0.1 - - [04/Apr/2020 02:57:47] "GET /manage_mentors HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 02:57:48] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:48] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:48] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:57:48] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:58:13] "GET /multiple_assign_page HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 02:58:13] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:58:13] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:58:13] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:58:13] "GET /static/images/icons/success_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 02:58:15] "GET /mentoring_joined HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 02:58:15] "GET /static/css/util.css 

127.0.0.1 - - [04/Apr/2020 03:01:40] "GET /static/images/icons/success_icon.png HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:01:41] "GET /mentoring_joined HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 03:01:41] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:01:41] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:01:41] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:01:41] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:01:41] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:01:43] "GET /multiple_assign_page HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 03:01:43] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:01:43] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:01:43] "GET /static/table2CSV.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:01:43] "GET /static/images/i

127.0.0.1 - - [04/Apr/2020 03:11:02] "GET /manage_mentors HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 03:11:02] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:11:03] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:11:03] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:11:03] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:11:17] "GET /manage_mentors HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 03:11:17] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:11:17] "GET /static/css/main.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:11:17] "GET /static/jquery.flexdatalist.min.js HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:11:17] "GET /static/jquery.flexdatalist.min.css HTTP/1.1" 304 -
127.0.0.1 - - [04/Apr/2020 03:11:44] "GET /manage_mentors HTTP/1.1" 200 -
127.0.0.1 - - [04/Apr/2020 03:11:44] "GET /static/css/util.css HTTP/1.1" 304 -
127.0.0.1

In [7]:
df = db_helper.readTableIntoDataFrame(connection,'professor')
df["Professor"] = df["lastname"].str.title().str.lstrip().str.rstrip()
del df["lastname"]
del df["firstname"]
df = df.rename(columns={"email":"Professor Email"})
col = ["ID","Professor","Professor Email"]
df=df[col]
df["Student Numbers"] = list(db_helper.getProfessorAndStudentNumberInDicionary(cursor).values())



In [8]:
list(db_helper.getProfessorAndStudentNumberInDicionary(cursor).values())

[106,
 62,
 71,
 59,
 72,
 73,
 70,
 65,
 48,
 27,
 41,
 69,
 64,
 64,
 71,
 67,
 69,
 0,
 14,
 12,
 13]

In [9]:
db_helper.getProfessorAndStudentNumberInDicionary(cursor)

{'berghel': 106,
 'datta': 62,
 'gewali': 71,
 'hall': 59,
 'jo': 72,
 'jorgensen': 73,
 'kim': 70,
 'larmore': 65,
 'minor': 48,
 'misch': 27,
 'nasoz': 41,
 'pedersen': 69,
 'stefik': 64,
 'taghva': 64,
 'yang': 71,
 'yfantis': 67,
 'zhan': 69,
 'cacho': 0,
 'vasko': 14,
 'dolly': 12,
 'chidella': 13}

In [50]:
df = db_helper.readTableIntoDataFrame(connection,'professor')
df

,ID,lastname,firstname,email
0,2,berghel,,hlb@acm.org
1,3,datta,,N/A
2,4,gewali,,laxmi.gewali@unlv.edu
3,5,hall,,guymon.hall@unlv.edu
4,6,jo,,juyeon.jo@unlv.edu
5,7,jorgensen,,jorgense@unlv.nevada.edu
6,8,kim,,yoohwan.kim@unlv.edu
7,9,larmore,,lawrence.larmore@unlv.edu
8,10,minor,,john.minor@unlv.edu
9,11,misch,,lee.misch@unlv.edu


In [11]:
db_helper.maxIdOfTable(cursor,'professor')

22

0

13